In [1]:
# Import necessary libraries

from operator import index

import pandas as pd

!pip install langdetect
import langdetect as detect

import numpy as np

import os

!pip install l
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

from google.colab import drive

# Download necessary data for natural language processing tasks

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=cd6931b54159a3ff0a42d97de39b918c26864fe3b6c21bd384942f99978859e4
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached vcversioner-2.16.0.0-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 6.8 MB/s eta 0:00:00
  Created wheel for l: filename=l-0.11.0-py3-none-any.whl size=8604 sha256=7e7bf8c2704d0689bf81b2ca1587a98443ad8574a531eed8e87da83fe0d6c077
  Stored in directory: /root/.cache/pip/wheels/ef/30/4b/417cc8adf3b2a3caa7d64bd4e617030e887e7a76daac82e122
  Created wheel for betterpath: filena

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
# Mount Google Drive to access data

drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
from google.colab import files  # Import the files module correctly
from IPython.display import display
import io
import pandas as pd

# Upload the file using the following code:
uploaded = files.upload()

# Get the filename and read the data into a Pandas DataFrame
filename = list(uploaded.keys())[0]  # Get the first uploaded file's name
df = pd.read_csv(io.BytesIO(uploaded[filename]), sep=',', header=0, on_bad_lines='skip', encoding="ISO-8859-1")

# Display the DataFrame to confirm it is loaded
display(df)


Saving spam.csv to spam (1).csv


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [7]:
# Drop any "Unnamed" columns

unnamed_cols = df.columns[df.columns.str.contains("Unnamed")]
df.drop(columns=unnamed_cols, inplace=True)

In [8]:
# Define regular expression variables to remove from SMS messages

reg_vars = ['http\S+', 'www\S+', 'https\S+', '\W\s+', '\d+', '\t+', '\d+', '\-+', '\\+', '\/+', '\"+', '\#+', '\++', '\@+', '\$+', '\%+', '\^+', '\&+', '\*+', '\(+', '\)+', '\[+', '\]+', '\{+', '\}+', '\|+', '\;+', '\:+', '\<+', '\>+', '\?+', '\,+', '\.+', '\=+', '\_+', '\~+', '\`+', '\s+']

df.replace(reg_vars, ' ', regex=True, inplace=True)

df.drop_duplicates(inplace=True)

df.replace('', np.nan, inplace=True)

df.dropna(inplace=True)

In [9]:
# Remove rows with non-ASCII characters from the dataframe

df = df[df['v2'].map(lambda x: x.isascii())]

In [11]:
# Drop non-English rows from the dataframe

for i in range(len(df)):
    try:
        ['v2'][i] = detect.detect(df['v2'][i])
        if df['v2'][i] != 'en':
            df.drop(i, inplace=True, index=False)
    except:
        pass

In [12]:
# Convert all the text data into lowercase

df['v2'] = df['v2'].astype(str).str.lower()

<ipython-input-12-e5af437bb2d0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['v2'] = df['v2'].astype(str).str.lower()


In [13]:
# Retrieve a list of English stop words and assign it to a var

stopwords = nltk.corpus.stopwords.words("english")

In [15]:
# Download necessary data for natural language processing tasks

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt_tab') # Download the punkt_tab data package

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [20]:
# Tokenize the 'v2' column and store it in a new column 'TokenSMS'
df.loc[:, 'TokenSMS'] = df['v2'].apply(lambda x: nltk.word_tokenize(x))

In [21]:
# Create a column containing the Tokenized words without the stopwords
df.loc[:, 'StopTokenSMS'] = df['TokenSMS'].apply(lambda x: [item for item in x if item not in stopwords])

In [22]:
# Create a column containing the StopTokenSMS text with words longer than 2 characters
df.loc[:, 'LengthTokenSMS'] = df['StopTokenSMS'].apply(lambda x: ' '.join([item for item in x if len(item) > 2]))

In [23]:
# Instantiate and the assigne a variable to the WordNetLemmatizer class

wordnet_lem = WordNetLemmatizer()

In [43]:
# Create a new column which contains the lemmatized words
df.loc[:, 'LemTokenSMS'] = df['LengthTokenSMS'].apply(wordnet_lem.lemmatize)

In [46]:
# Make a copy of the DataFrame to avoid modification on a view
df = df.copy()

# Clean all dataframes again
reg_vars = ['http\S+', 'www\S+', 'https\S+', '\W\s+', '\d+', '\t+', '\d+', '\-+', '\\+', '\/+', '\"+', '\#+', '\++', '\@+', '\$+', '\%+', '\^+', '\&+', '\*+', '\(+', '\)+', '\[+', '\]+', '\{+', '\}+', '\|+', '\;+', '\:+', '\<+', '\>+', '\?+', '\,+', '\.+', '\=+', '\_+', '\~+', '\`+', '\s+']

# Apply the replacements on the DataFrame
df.loc[:, :] = df.replace(reg_vars, ' ', regex=True)

# Replace empty strings with NaN
df.loc[:, :] = df.replace('', np.nan)

# Drop rows with NaN values
df.dropna(inplace=True)

In [47]:
# Initialize a CountVectorizer object

cv = CountVectorizer()

In [48]:
# fit_transform the data to a numpy array

x = cv.fit_transform(df['LemTokenSMS']).toarray()

In [49]:
x.shape

(4672, 6869)

In [50]:
# Replace 'spam' with 1 and 'ham' with 0
df['v1'] = df['v1'].replace({'spam': 1, 'ham': 0})

In [51]:
# Store the labels in  y

y = df['v1'].values

In [31]:
y.shape

(4672,)

In [32]:
# convert y to to int type

y = y.astype('int')

In [33]:
# Split the data into a training set and testing set

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [34]:
x_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [35]:
y_train.astype

<function ndarray.astype>

In [52]:
# Initialize a MultinomialNB objec

mnb=MultinomialNB()

In [53]:
# Training the classifier and making predictions on the test data

mnb.fit(x_train,y_train)
y_pred=mnb.predict(x_test)
print(accuracy_score(y_test,y_pred))

0.9689839572192513


In [38]:
import pickle

# Save the model to a file
with open('model.pkl', 'wb') as file:
    pickle.dump(mnb, file)

# Save the model to a file
with open('cv.pkl', 'wb') as file:
    pickle.dump(cv, file)

In [39]:
from google.colab import files

files.download('model.pkl')
files.download('cv.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
# Create a preprocessing function to process new text

def clean_dataframe(df):
    reg_vars = ['http\S+', 'www\S+', 'https\S+', '\W\s+', '\d+', '\t+', '\d+', '\-+', '\\+', '\/+', '\"+', '\#+', '\++', '\@+', '\$+', '\%+', '\^+', '\&+', '\*+', '\(+', '\)+', '\[+', '\]+', '\{+', '\}+', '\|+', '\;+', '\:+', '\<+', '\>+', '\?+', '\,+', '\.+', '\=+', '\_+', '\~+', '\`+', '\s+']
    df['text'].replace(reg_vars, ' ', regex=True, inplace=True)
    df['text'] = df['text'].astype(str).str.lower()
    df['text'] = df.apply(lambda column: nltk.word_tokenize(column['text']), axis=1)
    stopwords = nltk.corpus.stopwords.words('english')
    df['text'] = df['text'].apply(lambda x: [item for item in x if item not in stopwords])
    df['text'] = df['text'].apply(lambda x: ' '.join([item for item in x if len(item)>2]))
    df['text'] = df['text'].apply(wordnet_lem.lemmatize)


In [41]:
data = [{"text": "Urgent dont miss news dun say so early hor... U c already then say lucky man"}]

ndf = pd.DataFrame(data)

clean_dataframe(ndf)

<ipython-input-40-993951757459>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['text'].replace(reg_vars, ' ', regex=True, inplace=True)


In [42]:
newtext = cv.transform(ndf['text']).toarray()
prediction = mnb.predict(newtext)
print(prediction)

[0]
